In [0]:
#Training model 5000 (pure sine, same dimensions as disc groove)
import keras
import numpy as np
import os
import math
import random
from random import shuffle
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from skimage import io
from keras.utils import plot_model
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.io.wavfile
from contextlib import redirect_stdout
import json


#normalize unsigned 8-bits numpy array to [0;1]
def normalize_u8bits(data):
  data = data.astype(float)
  data /= 255.0
  return data

#normalize signed 16-bits numpy array to [-1;1]
def normalize_16bits(data):
  data = data.astype(float)
  data = (((data+32768)*(2))/65535)-1
  return data

#transform one np array as an array of overlapping smaller arrays of height time_steps
def window_reshape(data, time_steps, predictions):
  reshaped = []
  for i in range(0, data.shape[0]+1-time_steps, predictions):
    reshaped.append(data[i:i+time_steps,:])
  return np.array(reshaped)

#generate a perfect sine wave with amplitude range [-1;1] in a format ready for goal
def generate_sine(freq, size, offset, predictions):
  timestep = 1.0/104000.0
  sine = []
  block = []
  for i in range(size):
    amp = math.cos(2*math.pi*freq*(i+offset)*timestep)
    block.append(amp)
    if (i+1) % predictions == 0:
      sine.append(block)
      block = []
  return np.array(sine)

#plot two metrics and save in result folder
def plot_and_save(metric_val, metric_test, x_max, title, metric_name, time_step_name):
  length = len(metric_train)
  plt.plot(range(x_max), metric_val, label="validation results")
  plt.plot(range(x_max), metric_test, label="test results")
  plt.set_title(title)
  plt.set_ylabel(metric_name)
  plt.set_xlabel(time_step_name)
  plt.legend()
  plt.savefig(os.path.join(result_dir, title+'.png'))
  plt.close()

#create model
def create_model(filter_shapes, 
                 time_steps, 
                 img_width, 
                 activation_function, 
                 max_pool_shape, 
                 dropout_rate, 
                 predictions, 
                 padding,
                 learning_rate,
                 decay,
                 model_dir,
                 model_name):
  model = Sequential()
  model.add(Conv2D(32, filter_shapes[0],
                   input_shape=(time_steps, img_width, 1), padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(32, filter_shapes[1], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))   
  if not dropout_rate[0]==0.0:
    model.add(Dropout(dropout_rate[0]))
  
  model.add(Conv2D(64, filter_shapes[2], padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(64, filter_shapes[3], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))
  if not dropout_rate[1]==0.0:
    model.add(Dropout(dropout_rate[1]))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation(activation_function))
  if not dropout_rate[2]==0.0:
    model.add(Dropout(dropout_rate[2]))
  model.add(Dense(predictions))
  model.add(Activation('tanh'))
  
  opt = keras.optimizers.Adam(lr=learning_rate, decay=decay)

  model.compile(loss='mean_squared_error',
                optimizer=opt,
                metrics=['mean_absolute_error', 'mean_squared_error'])
            
  with open(os.path.join(model_dir, model_name+'_parameters.txt'), 'w') as f:
    f.write('time_steps_a : '+str(time_steps)+'\n')
    f.write('predictions : ' + str(predictions)+'\n')
    f.write('filter_shapes : '+str(filter_shapes)+'\n')
    f.write('max_pool_shape : '+str(max_pool_shape)+'\n')
    f.write('dropout_rate : '+str(dropout_rate)+'\n')
    f.write('activation : '+activation_function+'\n')
    f.write('padding : '+padding+'\n')
    f.write('learning_rate : '+str(learning_rate)+'\n')
    f.write('decay : '+str(decay)+'\n')
    f.write('optimizer : '+str(opt)+'\n')
  
  model_first_save(model, model_dir, model_name)
  return model
  
def model_first_save(model, model_dir, model_name):
  if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

  plot_model(model, to_file=os.path.join(model_dir, model_name+'_architecture.png'), show_shapes=True, show_layer_names=False)
  with open(os.path.join(model_dir, model_name+'_summary.txt'), 'w') as f:
    with redirect_stdout(f):
        model.summary()

def fit_model(model, images, time_steps, predictions, batch_size, epoch):
  counter=1
  for image_name in images:
    print('image '+str(counter)+'/'+str(len(images)))
    #print('name : '+image_name)
    counter+=1
    img = io.imread(os.path.join(img_train_dir, image_name) , as_gray=True)
    img = noising(img)
    img = img.reshape(img.shape[0], img.shape[1], 1)
    img_norm = normalize_u8bits(img)
    x_train = window_reshape(img_norm, time_steps, predictions)

    freq = int(image_name[1:image_name.find('_')])

    y_train = generate_sine(freq, x_train.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epoch,
                        initial_epoch=epoch-1,
                        shuffle=True,
                        validation_split=0.15,
                        verbose=1)
  return history
  
def test_model(model, time_steps, predictions):
  testing = os.listdir(img_test_dir)
  i = 0
  glob_scores = []
  for test in testing:
    img_test = io.imread(os.path.join(img_test_dir, test) , as_gray=True)
    #img_test = noising(img_test)
    img_test = img_test.reshape(img_test.shape[0], img_test.shape[1], 1)
    img_test_normalized = normalize_u8bits(img_test)
    x_test = window_reshape(img_test_normalized, time_steps, predictions)

    freq_test = int(test[1:test.find('_')])
    y_test = generate_sine(freq_test, x_test.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)
    
    scores = model.evaluate(x_test, y_test)
    #print(scores)
    glob_scores.append(scores)
  means = np.mean(glob_scores, axis=0)
  #print(means)
  
  return means


def noising(img, seed=None):
    
    random.seed(seed)
    #scratches generator
    for i in range(40000):
        h = random.randint(1, 3)
        w = random.randint(50, 150)
        y = random.randint(0, img.shape[0]+1)
        x = random.randint(0, img.shape[1]+1)

        img[y-h//2:y+h//2, x-w//2:x+w//2] = 0
        
    #holes generator
    for i in range(8000):
        h = random.randint(5, 15)
        w = random.randint(10, 100)
        y = random.randint(0, img.shape[0]+1)
        x = random.randint(0, img.shape[1]+1)

        img[y-h//2:y+h//2, x-w//2:x+w//2] = 0
        
    np.random.seed(seed)
    gauss = np.random.normal(-100.0,50.0,img.shape)
    noisy = img + gauss
    clipped = np.clip(noisy, 0, 255)
    
    return np.around(clipped).astype(np.uint8)

  
#-----------MAIN----------------
#dir paths
models_dir = '/content/drive/My Drive/models'
results_dir = '/content/drive/My Drive/results'
img_train_dir = "/content/drive/My Drive/disc_sim_data/train"
img_test_dir = "/content/drive/My Drive/disc_sim_data/noisy_test"

model_version_start = 5000 #for output numbering

#network hyper-parameters, default values
epoch_start = 2
epochs = 20
batch_size = 32
time_steps = 15
predictions = 1
filter_shapes = [(3,3),(3,3),(3,3),(3,3)]
img_width = 320
learning_rate = 0.001
decay = 0.0
steps_for_testing = 100 # 1 step = 1 image
max_pool_shape = (2, 2)
dropout_rate = [0.25, 0.25, 0.5]
activation_function = 'relu'
padding='same'
division = 5 # epoch division for testing and saving

current_epoch = 1
current_mv = 0

for epoch in range(epoch_start,epochs+epoch_start):
  
  model_version = model_version_start
  
  images = os.listdir(img_train_dir)
  shuffle(images)
  
  model_name = 'model'+str(model_version)

  print(model_name+', epoch '+str(epoch))
  
  model_dir = os.path.join(models_dir, model_name)
  if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

  #loading model
  if epoch > 1:
    model = load_model(os.path.join(model_dir, model_name+'_epoch'+str(epoch-1)+'.h5'))
  else:

    model = create_model(filter_shapes=filter_shapes,
                        time_steps=time_steps,
                        img_width=img_width,
                        activation_function=activation_function,
                        max_pool_shape=max_pool_shape,
                        dropout_rate=dropout_rate,
                        predictions=predictions,
                        padding=padding,
                        learning_rate = learning_rate,
                        decay = decay,
                        model_dir = model_dir,
                        model_name = model_name)
  size = len(images)//division
  for nth in range(0, division) :
    #fitting model over one epoch
    train_results = fit_model(model, images[nth*size:(nth+1)*size], time_steps, predictions, batch_size, epoch)

    #saving model
    stamp = str(epoch-1)+'.'+str(nth+1) if nth < division-1 else str(epoch)
    model.save(os.path.join(model_dir, model_name+'_epoch'+stamp+'.h5'))

    #testing model
    test_results = test_model(model, time_steps, predictions)

    #appending results
    result_dir = os.path.join(results_dir, model_name)
    if epoch==1 and nth == 0:
      results = {}
      results['mae_validation']={}
      results['mse_validation']={}
      results['mae_test']={}
      results['mse_test']={}
      if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    else:
      with open(os.path.join(result_dir, model_name+'_results.json'), 'r') as fp:
        results = json.load(fp)

    results['mae_validation'][stamp] = train_results.history['val_mean_absolute_error']
    results['mse_validation'][stamp] = train_results.history['val_mean_squared_error']
    results['mae_test'][stamp] = test_results[1]
    results['mse_test'][stamp] = test_results[0]

    with open(os.path.join(result_dir, model_name+'_results.json'), 'w') as f:
      json.dump(results, f)

In [9]:
#Training model 5002 (disc groove, sound from tape)
import keras
import numpy as np
import os
import math
import random
from random import shuffle
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from skimage import io
from keras.utils import plot_model
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.io.wavfile
from contextlib import redirect_stdout
import json
import xml.etree.ElementTree as ET
from scipy.io.wavfile import read


#normalize unsigned 8-bits numpy array to [0;1]
def normalize_u8bits(data):
  data = data.astype(float)
  data /= 255.0
  return data

#normalize signed 16-bits numpy array to [-1;1]
def normalize_16bits(data):
  data = data.astype(float)
  data = (((data+32768)*(2))/65535)-1
  return data

#transform one disc image as an array of overlapping smaller arrays of height time_steps
def window_reshape(data, time_steps, predictions, track, window_width):
  reshaped = []
  for i in range(0, len(track), predictions):
    #print('Window at '+str(track[i][0:2]))
    top_bound = max(track[i][0]-time_steps//2, 0)
    bottom_bound = min(track[i][0]+time_steps//2 + (1 if time_steps%2==1 else 0), data.shape[0])
    left_bound = max(track[i][1]-window_width//2, 0)
    right_bound = min(track[i][1]+window_width//2 + (1 if window_width%2==1 else 0), data.shape[1])
    window = data[top_bound:bottom_bound, left_bound:right_bound]

    padding=((max(0, time_steps//2-track[i][0]), 
                max(0, track[i][0]+time_steps//2-data.shape[0]+(1 if time_steps%2==1 else 0))),
               (max(0, window_width//2-track[i][1]), 
                max(0, track[i][1]+window_width//2-data.shape[1]+(1 if window_width%2==1 else 0))),
                (0,0))
    if window.shape[0] < time_steps or window.shape[1] < window_width:
      #print('too small : ' + str(window.shape))
      #print('adding : '+ str(padding))
      window = np.pad(window, padding, 'edge')
      #print('result : '+str(padded.shape))
      #print(str(padded[:,:10]))
      #input()
    reshaped.append(window)
    
  return np.array(reshaped)


#plot two metrics and save in result folder
def plot_and_save(metric_val, metric_test, x_max, title, metric_name, time_step_name):
  length = len(metric_train)
  plt.plot(range(x_max), metric_val, label="validation results")
  plt.plot(range(x_max), metric_test, label="test results")
  plt.set_title(title)
  plt.set_ylabel(metric_name)
  plt.set_xlabel(time_step_name)
  plt.legend()
  plt.savefig(os.path.join(result_dir, title+'.png'))
  plt.close()

#create model
def create_model(filter_shapes, 
                 time_steps, 
                 img_width, 
                 activation_function, 
                 max_pool_shape, 
                 dropout_rate, 
                 predictions, 
                 padding,
                 learning_rate,
                 decay,
                 model_dir,
                 model_name):
  model = Sequential()
  model.add(Conv2D(32, filter_shapes[0],
                   input_shape=(time_steps, img_width, 1), padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(32, filter_shapes[1], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))   
  if not dropout_rate[0]==0.0:
    model.add(Dropout(dropout_rate[0]))
  
  model.add(Conv2D(64, filter_shapes[2], padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(64, filter_shapes[3], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))
  if not dropout_rate[1]==0.0:
    model.add(Dropout(dropout_rate[1]))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation(activation_function))
  if not dropout_rate[2]==0.0:
    model.add(Dropout(dropout_rate[2]))
  model.add(Dense(predictions))
  model.add(Activation('tanh'))
  
  opt = keras.optimizers.Adam(lr=learning_rate, decay=decay)

  model.compile(loss='mean_squared_error',
                optimizer=opt,
                metrics=['mean_absolute_error', 'mean_squared_error'])
            
  with open(os.path.join(model_dir, model_name+'_parameters.txt'), 'w') as f:
    f.write('time_steps_a : '+str(time_steps)+'\n')
    f.write('predictions : ' + str(predictions)+'\n')
    f.write('filter_shapes : '+str(filter_shapes)+'\n')
    f.write('max_pool_shape : '+str(max_pool_shape)+'\n')
    f.write('dropout_rate : '+str(dropout_rate)+'\n')
    f.write('activation : '+activation_function+'\n')
    f.write('padding : '+padding+'\n')
    f.write('learning_rate : '+str(learning_rate)+'\n')
    f.write('decay : '+str(decay)+'\n')
    f.write('optimizer : '+str(opt)+'\n')
  
  model_first_save(model, model_dir, model_name)
  return model
  
def model_first_save(model, model_dir, model_name):
  if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

  plot_model(model, to_file=os.path.join(model_dir, model_name+'_architecture.png'), show_shapes=True, show_layer_names=False)
  with open(os.path.join(model_dir, model_name+'_summary.txt'), 'w') as f:
    with redirect_stdout(f):
        model.summary()

def fit_model(model, images, time_steps, predictions, batch_size, epoch):
  counter=1  
  for image in images:
    print('image '+str(counter)+'/'+str(len(images)))
    #print('name : '+image_name)
    counter+=1
    if not image.endswith('.tif'):
      continue
    #read image
    groove = io.imread(os.path.join(data_dir, image), as_gray=True)
    groove = groove.reshape(groove.shape[0], groove.shape[1], 1)
    groove = normalize_u8bits(groove)

    #read track
    track_data = ET.parse(os.path.join(data_dir, image.replace('.tif', '.track')))
    root = track_data.getroot()
    track = []
    for x in root.findall('ArrayOfDouble'):
      point = []
      for d in x.findall('double'):
        point.append(int(d.text))
      track.append(point)  
    track.pop(0)

    #read sound
    samplerate, sound=read(os.path.join(data_dir, image.replace('.tif', '.wav')))
    sound = np.array(sound[0:len(track)])
    sound = normalize_16bits(sound)

    for i in range(0, len(track), 50000):
      x_data = window_reshape(groove, time_steps, predictions, track[i:min(len(track), i+50000)], window_width)
      y_data = sound[i:min(len(track), i+50000)]
      history = model.fit(x_data, y_data, 
                batch_size=32, 
                epochs=epoch,
                initial_epoch=epoch-1,
                shuffle=True,
                validation_split=0.15,
                verbose=1)
  return history
  
def test_model(model, time_steps, predictions):
  testing = os.listdir(img_test_dir)
  i = 0
  glob_scores = []
  for test in testing:
    img_test = io.imread(os.path.join(img_test_dir, test) , as_gray=True)
    #img_test = noising(img_test)
    img_test = img_test.reshape(img_test.shape[0], img_test.shape[1], 1)
    img_test_normalized = normalize_u8bits(img_test)
    x_test = window_reshape(img_test_normalized, time_steps, predictions)

    freq_test = int(test[1:test.find('_')])
    y_test = generate_sine(freq_test, x_test.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)
    
    scores = model.evaluate(x_test, y_test)
    #print(scores)
    glob_scores.append(scores)
  means = np.mean(glob_scores, axis=0)
  #print(means)
  
  return means

  
#-----------MAIN----------------
#dir paths
models_dir = '/content/drive/My Drive/models'
results_dir = '/content/drive/My Drive/results'
data_dir = "/content/drive/My Drive/disc_data"


model_version_start = 6000 #for output numbering

#network hyper-parameters, default values
epoch_start = 1
epochs = 20
batch_size = 32
time_steps = 21
predictions = 1
filter_shapes = [(3,3),(3,3),(3,3),(3,3)]
img_width = 360
window_width = 360
learning_rate = 0.0005
decay = 0
steps_for_testing = 100 # 1 step = 1 image
max_pool_shape = (2, 2)
dropout_rate = [0.5, 0.5, 0.7]
activation_function = 'relu'
padding='same'
division = 1 # epoch division for testing and saving

current_epoch = 1
current_mv = 0

for epoch in range(epoch_start,epochs+epoch_start):
  
  model_version = model_version_start
  
  images_raw = os.listdir(data_dir)
  images=[]
  for image in images_raw:
    if image.endswith('.tif'):
      images.append(image)
  
  shuffle(images)
  model_name = 'model'+str(model_version)

  print(model_name+', epoch '+str(epoch))
  
  model_dir = os.path.join(models_dir, model_name)
  if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

  #loading model
  if epoch > 1:
    model = load_model(os.path.join(model_dir, model_name+'_epoch'+str(epoch-1)+'.h5'))
  else:
    model = create_model(filter_shapes=filter_shapes,
                        time_steps=time_steps,
                        img_width=img_width,
                        activation_function=activation_function,
                        max_pool_shape=max_pool_shape,
                        dropout_rate=dropout_rate,
                        predictions=predictions,
                        padding=padding,
                        learning_rate = learning_rate,
                        decay = decay,
                        model_dir = model_dir,
                        model_name = model_name)
  size = len(images)//division
  for nth in range(0, division) :
    #fitting model over one epoch
    train_results = fit_model(model, images[nth*size:(nth+1)*size], time_steps, predictions, batch_size, epoch)

    #saving model
    stamp = str(epoch-1)+'.'+str(nth+1) if nth < division-1 else str(epoch)
    model.save(os.path.join(model_dir, model_name+'_epoch'+stamp+'.h5'))

    #testing model
    #test_results = test_model(model, time_steps, predictions)

    #appending results
    result_dir = os.path.join(results_dir, model_name)
    if epoch==1 and nth == 0:
      results = {}
      results['mae_validation']={}
      results['mse_validation']={}
      #results['mae_test']={}
      #results['mse_test']={}
      if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    else:
      with open(os.path.join(result_dir, model_name+'_results.json'), 'r') as fp:
        results = json.load(fp)

    results['mae_validation'][stamp] = train_results.history['val_mean_absolute_error']
    results['mse_validation'][stamp] = train_results.history['val_mean_squared_error']
    #results['mae_test'][stamp] = test_results[1]
    #results['mse_test'][stamp] = test_results[0]

    with open(os.path.join(result_dir, model_name+'_results.json'), 'w') as f:
      json.dump(results, f)

model5005, epoch 1
image 1/1


/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Train on 42500 samples, validate on 7500 samples
Epoch 1/1
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0227 - mean_absolute_error: 0.1198 - mean_squared_error: 0.0227 - val_loss: 0.0140 - val_mean_absolute_error: 0.0953 - val_mean_squared_error: 0.0140
Train on 42500 samples, validate on 7500 samples
Epoch 1/1
42500/42500 [==============================] - 84s 2ms/step - loss: 0.0135 - mean_absolute_error: 0.0948 - mean_squared_error: 0.0135 - val_loss: 0.0149 - val_mean_absolute_error: 0.1019 - val_mean_squared_error: 0.0149
Train on 42500 samples, validate on 7500 samples
Epoch 1/1
42500/42500 [==============================] - 84s 2ms/step - loss: 0.0067 - mean_absolute_error: 0.0652 - mean_squared_error: 0.0067 - val_loss: 0.0029 - val_mean_absolute_error: 0.0451 - val_mean_squared_error: 0.0029
Train on 42500 samples, validate on 7500 samples
Epoch 1/1
42500/42500 [==============================] - 84s 2ms/step - loss: 0.0227 - mean_absolute_error: 0.1039

KeyboardInterrupt: ignored

In [2]:
#Finding best learning rate for Adam
import keras
import numpy as np
import os
import math
import random
from random import shuffle
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from skimage import io
from keras.utils import plot_model
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.io.wavfile
from contextlib import redirect_stdout
import json
import xml.etree.ElementTree as ET
from scipy.io.wavfile import read


#normalize unsigned 8-bits numpy array to [0;1]
def normalize_u8bits(data):
  data = data.astype(float)
  data /= 255.0
  return data

#normalize signed 16-bits numpy array to [-1;1]
def normalize_16bits(data):
  data = data.astype(float)
  data = (((data+32768)*(2))/65535)-1
  return data

#transform one disc image as an array of overlapping smaller arrays of height time_steps
def window_reshape(data, time_steps, predictions, track, window_width):
  reshaped = []
  for i in range(0, len(track), predictions):
    #print('Window at '+str(track[i][0:2]))
    top_bound = max(track[i][0]-time_steps//2, 0)
    bottom_bound = min(track[i][0]+time_steps//2 + (1 if time_steps%2==1 else 0), data.shape[0])
    left_bound = max(track[i][1]-window_width//2, 0)
    right_bound = min(track[i][1]+window_width//2 + (1 if window_width%2==1 else 0), data.shape[1])
    window = data[top_bound:bottom_bound, left_bound:right_bound]

    padding=((max(0, time_steps//2-track[i][0]), 
                max(0, track[i][0]+time_steps//2-data.shape[0]+(1 if time_steps%2==1 else 0))),
               (max(0, window_width//2-track[i][1]), 
                max(0, track[i][1]+window_width//2-data.shape[1]+(1 if window_width%2==1 else 0))),
                (0,0))
    if window.shape[0] < time_steps or window.shape[1] < window_width:
      #print('too small : ' + str(window.shape))
      #print('adding : '+ str(padding))
      window = np.pad(window, padding, 'edge')
      #print('result : '+str(padded.shape))
      #print(str(padded[:,:10]))
      #input()
    reshaped.append(window)
    
  return np.array(reshaped)


#plot two metrics and save in result folder
def plot_and_save(metric_val, metric_test, x_max, title, metric_name, time_step_name):
  length = len(metric_train)
  plt.plot(range(x_max), metric_val, label="validation results")
  plt.plot(range(x_max), metric_test, label="test results")
  plt.set_title(title)
  plt.set_ylabel(metric_name)
  plt.set_xlabel(time_step_name)
  plt.legend()
  plt.savefig(os.path.join(result_dir, title+'.png'))
  plt.close()

#create model
def create_model(filter_shapes, 
                 time_steps, 
                 img_width, 
                 activation_function, 
                 max_pool_shape, 
                 dropout_rate, 
                 predictions, 
                 padding,
                 learning_rate,
                 decay,
                 model_dir,
                 model_name):
  model = Sequential()
  model.add(Conv2D(32, filter_shapes[0],
                   input_shape=(time_steps, img_width, 1), padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(32, filter_shapes[1], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))   
  if not dropout_rate[0]==0.0:
    model.add(Dropout(dropout_rate[0]))
  
  model.add(Conv2D(64, filter_shapes[2], padding=padding))
  model.add(Activation(activation_function))
  model.add(Conv2D(64, filter_shapes[3], padding=padding))
  model.add(Activation(activation_function))
  if not max_pool_shape == (1,1):
      model.add(MaxPooling2D(pool_size=max_pool_shape))
  if not dropout_rate[1]==0.0:
    model.add(Dropout(dropout_rate[1]))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation(activation_function))
  if not dropout_rate[2]==0.0:
    model.add(Dropout(dropout_rate[2]))
  model.add(Dense(predictions))
  model.add(Activation('tanh'))
  
  opt = keras.optimizers.Adam(lr=learning_rate, decay=decay)

  model.compile(loss='mean_squared_error',
                optimizer=opt,
                metrics=['mean_absolute_error', 'mean_squared_error'])
            
  with open(os.path.join(model_dir, model_name+'_parameters.txt'), 'w') as f:
    f.write('time_steps_a : '+str(time_steps)+'\n')
    f.write('predictions : ' + str(predictions)+'\n')
    f.write('filter_shapes : '+str(filter_shapes)+'\n')
    f.write('max_pool_shape : '+str(max_pool_shape)+'\n')
    f.write('dropout_rate : '+str(dropout_rate)+'\n')
    f.write('activation : '+activation_function+'\n')
    f.write('padding : '+padding+'\n')
    f.write('learning_rate : '+str(learning_rate)+'\n')
    f.write('decay : '+str(decay)+'\n')
    f.write('optimizer : '+str(opt)+'\n')
  
  model_first_save(model, model_dir, model_name)
  return model
  
def model_first_save(model, model_dir, model_name):
  if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

  plot_model(model, to_file=os.path.join(model_dir, model_name+'_architecture.png'), show_shapes=True, show_layer_names=False)
  with open(os.path.join(model_dir, model_name+'_summary.txt'), 'w') as f:
    with redirect_stdout(f):
        model.summary()

def fit_model(model, images, time_steps, predictions, batch_size, epoch):
  counter=1  
  for image in images:
    print('image '+str(counter)+'/'+str(len(images)))
    #print('name : '+image_name)
    counter+=1
    if not image.endswith('.tif'):
      continue
    #read image
    groove = io.imread(os.path.join(data_dir, image), as_gray=True)
    groove = groove.reshape(groove.shape[0], groove.shape[1], 1)
    groove = normalize_u8bits(groove)

    #read track
    track_data = ET.parse(os.path.join(data_dir, image.replace('.tif', '.track')))
    root = track_data.getroot()
    track = []
    for x in root.findall('ArrayOfDouble'):
      point = []
      for d in x.findall('double'):
        point.append(int(d.text))
      track.append(point)  
    track.pop(0)

    #read sound
    samplerate, sound=read(os.path.join(data_dir, image.replace('.tif', '.wav')))
    sound = np.array(sound[0:len(track)])
    sound = normalize_16bits(sound)

    for i in range(0, len(track), 50000):
      x_data = window_reshape(groove, time_steps, predictions, track[i:min(len(track), i+50000)], window_width)
      y_data = sound[i:min(len(track), i+50000)]
      history = model.fit(x_data, y_data, 
                batch_size=32, 
                epochs=epoch,
                initial_epoch=epoch-1,
                shuffle=True,
                validation_split=0.15,
                verbose=1)
  return history
  
def test_model(model, time_steps, predictions):
  testing = os.listdir(img_test_dir)
  i = 0
  glob_scores = []
  for test in testing:
    img_test = io.imread(os.path.join(img_test_dir, test) , as_gray=True)
    #img_test = noising(img_test)
    img_test = img_test.reshape(img_test.shape[0], img_test.shape[1], 1)
    img_test_normalized = normalize_u8bits(img_test)
    x_test = window_reshape(img_test_normalized, time_steps, predictions)

    freq_test = int(test[1:test.find('_')])
    y_test = generate_sine(freq_test, x_test.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)
    
    scores = model.evaluate(x_test, y_test)
    #print(scores)
    glob_scores.append(scores)
  means = np.mean(glob_scores, axis=0)
  #print(means)
  
  return means

  
#-----------MAIN----------------
#dir paths
models_dir = '/content/drive/My Drive/models'
results_dir = '/content/drive/My Drive/results'
data_dir = "/content/drive/My Drive/disc_data"


model_version_start = 6001 #for output numbering

#network hyper-parameters, default values
epoch_start = 4
epochs = 20
batch_size = 32
time_steps = 21
predictions = 1
filter_shapes = [(3,3),(3,3),(3,3),(3,3)]
img_width = 360
window_width = 360
learning_rate = 0.0005
decay = 0
steps_for_testing = 100 # 1 step = 1 image
max_pool_shape = (2, 2)
dropout_rate = [0.25, 0.5, 0.5]
activation_function = 'relu'
padding='same'
division = 1 # epoch division for testing and saving

current_epoch = 1
current_mv = 0

for epoch in range(epoch_start,epochs+epoch_start):
  
  model_version = model_version_start
  
  images_raw = os.listdir(data_dir)
  images=[]
  for image in images_raw:
    if image.endswith('.tif'):
      images.append(image)
  
  shuffle(images)
  
  for learning_rate in [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]:
    
    model_name = 'model'+str(model_version)

    print(model_name+', epoch '+str(epoch))

    model_dir = os.path.join(models_dir, model_name)
    if not os.path.isdir(model_dir):
      os.makedirs(model_dir)

    #loading model
    if epoch > 1:
      model = load_model(os.path.join(model_dir, model_name+'_epoch'+str(epoch-1)+'.h5'))
    else:
      model = create_model(filter_shapes=filter_shapes,
                          time_steps=time_steps,
                          img_width=img_width,
                          activation_function=activation_function,
                          max_pool_shape=max_pool_shape,
                          dropout_rate=dropout_rate,
                          predictions=predictions,
                          padding=padding,
                          learning_rate = learning_rate,
                          decay = decay,
                          model_dir = model_dir,
                          model_name = model_name)
    size = len(images)//division
    for nth in range(0, division) :
      #fitting model over one epoch
      train_results = fit_model(model, images[nth*size:(nth+1)*size], time_steps, predictions, batch_size, epoch)

      #saving model
      stamp = str(epoch-1)+'.'+str(nth+1) if nth < division-1 else str(epoch)
      model.save(os.path.join(model_dir, model_name+'_epoch'+stamp+'.h5'))

      #testing model
      #test_results = test_model(model, time_steps, predictions)

      #appending results
      result_dir = os.path.join(results_dir, model_name)
      if epoch==1 and nth == 0:
        results = {}
        results['mae_validation']={}
        results['mse_validation']={}
        #results['mae_test']={}
        #results['mse_test']={}
        if not os.path.isdir(result_dir):
          os.makedirs(result_dir)
      else:
        with open(os.path.join(result_dir, model_name+'_results.json'), 'r') as fp:
          results = json.load(fp)

      results['mae_validation'][stamp] = train_results.history['val_mean_absolute_error']
      results['mse_validation'][stamp] = train_results.history['val_mean_squared_error']
      #results['mae_test'][stamp] = test_results[1]
      #results['mse_test'][stamp] = test_results[0]

      with open(os.path.join(result_dir, model_name+'_results.json'), 'w') as f:
        json.dump(results, f)
    model_version += 1

Using TensorFlow backend.


model6001, epoch 4


W0809 07:01:44.047894 140128243148672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 07:01:44.105293 140128243148672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0809 07:01:44.175969 140128243148672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0809 07:01:44.184390 140128243148672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 07:01:44.190922 140128243148672 deprecation_wrap

image 1/1


/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


...

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 87s 2ms/step - loss: 0.0239 - mean_absolute_error: 0.1242 - mean_squared_error: 0.0239 - val_loss: 0.0140 - val_mean_absolute_error: 0.0954 - val_mean_squared_error: 0.0140
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0157 - mean_absolute_error: 0.1015 - mean_squared_error: 0.0157 - val_loss: 0.0149 - val_mean_absolute_error: 0.1017 - val_mean_squared_error: 0.0149
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0093 - mean_absolute_error: 0.0769 - mean_squared_error: 0.0093 - val_loss: 0.0029 - val_mean_absolute_error: 0.0453 - val_mean_squared_error: 0.0029
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 87s 2ms/step - loss: 0.0252 - mean_absolute_error: 0.1119

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 87s 2ms/step - loss: 0.0218 - mean_absolute_error: 0.1186 - mean_squared_error: 0.0218 - val_loss: 0.0140 - val_mean_absolute_error: 0.0952 - val_mean_squared_error: 0.0140
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0136 - mean_absolute_error: 0.0951 - mean_squared_error: 0.0136 - val_loss: 0.0150 - val_mean_absolute_error: 0.1021 - val_mean_squared_error: 0.0150
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0068 - mean_absolute_error: 0.0658 - mean_squared_error: 0.0068 - val_loss: 0.0029 - val_mean_absolute_error: 0.0452 - val_mean_squared_error: 0.0029
Train on 42500 samples, validate on 7500 samples
Epoch 7/7
42500/42500 [==============================] - 85s 2ms/step - loss: 0.0228 - mean_absolute_error: 0.1041

KeyboardInterrupt: ignored

In [1]:
#Using model and real disc images
import keras
import numpy as np
import os
import math
import random
from random import shuffle
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from skimage import io
from keras.utils import plot_model
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.io.wavfile
from contextlib import redirect_stdout
import json
import xml.etree.ElementTree as ET
import PIL.Image as Image


#normalize unsigned 8-bits numpy array to [0;1]
def normalize_u8bits(data):
  data = data.astype(float)
  data /= 255.0
  return data

#normalize signed 16-bits numpy array to [-1;1]
def normalize_16bits(data):
  data = data.astype(float)
  data = (((data+32768)*(2))/65535)-1
  return data

#transform one disc image as an array of overlapping smaller arrays of height time_steps
def window_reshape(data, time_steps, predictions, track, window_width):
  reshaped = []
  for i in range(0, len(track), predictions):
    #print('Window at '+str(track[i][0:2]))
    top_bound = max(track[i][0]-time_steps//2, 0)
    bottom_bound = min(track[i][0]+time_steps//2 + (1 if time_steps%2==1 else 0), data.shape[0])
    left_bound = max(track[i][1]-window_width//2, 0)
    right_bound = min(track[i][1]+window_width//2 + (1 if window_width%2==1 else 0), data.shape[1])
    window = data[top_bound:bottom_bound, left_bound:right_bound]

    padding=((max(0, time_steps//2-track[i][0]), 
                max(0, track[i][0]+time_steps//2-data.shape[0]+(1 if time_steps%2==1 else 0))),
               (max(0, window_width//2-track[i][1]), 
                max(0, track[i][1]+window_width//2-data.shape[1]+(1 if window_width%2==1 else 0))),
                (0,0))
    if window.shape[0] < time_steps or window.shape[1] < window_width:
      #print('too small : ' + str(window.shape))
      #print('adding : '+ str(padding))
      window = np.pad(window, padding, 'edge')
      #print('result : '+str(padded.shape))
      #print(str(padded[:,:10]))
      #input()
    reshaped.append(window)
  return np.array(reshaped)


#plot two metrics and save in result folder
def plot_and_save(metric_val, metric_test, x_max, title, metric_name, time_step_name):
  length = len(metric_train)
  plt.plot(range(x_max), metric_val, label="validation results")
  plt.plot(range(x_max), metric_test, label="test results")
  plt.set_title(title)
  plt.set_ylabel(metric_name)
  plt.set_xlabel(time_step_name)
  plt.legend()
  plt.savefig(os.path.join(result_dir, title+'.png'))
  plt.close()
  
def test_model(test_images, model, time_steps, predictions, window_width):
  testing = test_images
  i = 0
  glob_scores = []
  for test in testing:
    img_test = io.imread(os.path.join(img_test_dir, test) , as_gray=True)
    img_test = img_test.reshape(img_test.shape[0], img_test.shape[1], 1)
    img_test_normalized = normalize_u8bits(img_test)
    x_test = window_reshape(img_test_normalized, time_steps, predictions)

    freq_test = int(test[1:test.find('_')])
    y_test = generate_sine(freq_test, x_test.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)
    
    scores = model.evaluate(x_test, y_test)
    #print(scores)
    glob_scores.append(scores)
  means = np.mean(glob_scores, axis=0)
  #print(means)
  
  return means


#-----------MAIN----------------
#dir paths
models_dir = '/content/drive/My Drive/models'
results_dir = '/content/drive/My Drive/results'
data_dir = "/content/drive/My Drive/disc_data"

model_version = 5004
model_epoch = 1

#network hyper-parameters, default values
time_steps = 15
predictions = 1
window_width = 360
  
model_name = 'model'+str(model_version)
res_dir = os.path.join(results_dir, model_name)
if not os.path.isdir(res_dir):
  os.makedirs(res_dir)

for model_epoch in range(13, 14):
  print('epoch : '+str(model_epoch))
  model_weights_name = model_name+'_epoch'+str(model_epoch)+'.h5'
  model = load_model(os.path.join(models_dir, model_name, model_weights_name))
  
  images = os.listdir(data_dir)
  for image in images:
    if image.endswith('.tif'):
      groove = io.imread(os.path.join(data_dir, image), as_gray=True)
      groove = groove.reshape(groove.shape[0], groove.shape[1], 1)
      groove = normalize_u8bits(groove)

      track_data = ET.parse(os.path.join(data_dir, image.replace('.tif', '.track')))
      root = track_data.getroot()
      track = []
      for x in root.findall('ArrayOfDouble'):
        point = []
        for d in x.findall('double'):
          point.append(int(d.text))
        track.append(point)  
      track.pop(0)

      total_pred = np.empty((len(track), 2))

      for i in range(0, len(track), 50000):
        print('starting block : '+str(i))
        x_data = window_reshape(groove, time_steps, predictions, track[i:min(len(track), i+50000)], window_width)
        window=x_data[0]*255 
        #print(window.shape)
        window=np.squeeze(window)
        window = np.array(window)
        window = window.astype('uint8')
        im = Image.fromarray(window)
        im.save(os.path.join(res_dir, image.replace('.tif','_point_'+str(track[i])+'.tif')))
        prediction = model.predict(x_data)
        prediction = prediction.astype(float)
        pred_audio = np.column_stack((prediction, prediction))
        for index, line in enumerate(pred_audio):
          total_pred[i+index] = line
        #print(pred_audio.shape)
        #total_pred = np.append(total_pred, pred_audio)
        #print(total_pred.shape)
      
      scipy.io.wavfile.write(
          os.path.join(
              res_dir, 
              image+'_model'+str(model_version)+'_epoch'+str(model_epoch)+'_pred.wav'), 104000, total_pred)

Using TensorFlow backend.
W0808 10:35:51.399158 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 10:35:51.441619 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 10:35:51.493452 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0808 10:35:51.500994 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 10:35:51.502275 140294

epoch : 13


W0808 10:35:52.061638 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0808 10:35:55.494295 140294032181120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



starting block : 0
starting block : 50000
starting block : 100000
starting block : 150000
starting block : 200000
starting block : 250000
starting block : 300000
starting block : 350000
starting block : 400000
starting block : 450000
starting block : 500000
starting block : 550000


In [3]:
#transfer learning from model 5000
import keras
import numpy as np
import os
import math
import random
from random import shuffle
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from skimage import io
from keras.utils import plot_model
from matplotlib import pyplot as plt
from IPython.display import clear_output
import scipy.io.wavfile
from contextlib import redirect_stdout
import json
import xml.etree.ElementTree as ET
from scipy.io.wavfile import read


#normalize unsigned 8-bits numpy array to [0;1]
def normalize_u8bits(data):
  data = data.astype(float)
  data /= 255.0
  return data

#normalize signed 16-bits numpy array to [-1;1]
def normalize_16bits(data):
  data = data.astype(float)
  data = (((data+32768)*(2))/65535)-1
  return data

#normalize signed 32-bits numpy array to [-1;1]
def normalize_32bits(data):
  data = data.astype(float)
  data = (((data+2147483648)*(2))/4294967295)-1
  return data

#transform one disc image as an array of overlapping smaller arrays of height time_steps
def window_reshape(data, time_steps, predictions, track, window_width):
  reshaped = []
  for i in range(0, len(track), predictions):
    #print('Window at '+str(track[i][0:2]))
    top_bound = max(track[i][0]-time_steps//2, 0)
    bottom_bound = min(track[i][0]+time_steps//2 + (1 if time_steps%2==1 else 0), data.shape[0])
    left_bound = max(track[i][1]-window_width//2, 0)
    right_bound = min(track[i][1]+window_width//2 + (1 if window_width%2==1 else 0), data.shape[1])
    window = data[top_bound:bottom_bound, left_bound:right_bound]

    padding=((max(0, time_steps//2-track[i][0]), 
                max(0, track[i][0]+time_steps//2-data.shape[0]+(1 if time_steps%2==1 else 0))),
               (max(0, window_width//2-track[i][1]), 
                max(0, track[i][1]+window_width//2-data.shape[1]+(1 if window_width%2==1 else 0))),
                (0,0))
    if window.shape[0] < time_steps or window.shape[1] < window_width:
      #print('too small : ' + str(window.shape))
      #print('adding : '+ str(padding))
      window = np.pad(window, padding, 'edge')
      #print('result : '+str(padded.shape))
      #print(str(padded[:,:10]))
      #input()
    reshaped.append(window)
    
  return np.array(reshaped)


#plot two metrics and save in result folder
def plot_and_save(metric_val, metric_test, x_max, title, metric_name, time_step_name):
  length = len(metric_train)
  plt.plot(range(x_max), metric_val, label="validation results")
  plt.plot(range(x_max), metric_test, label="test results")
  plt.set_title(title)
  plt.set_ylabel(metric_name)
  plt.set_xlabel(time_step_name)
  plt.legend()
  plt.savefig(os.path.join(result_dir, title+'.png'))
  plt.close()
  
def test_model(test_images, model, time_steps, predictions, window_width):
  testing = test_images
  i = 0
  glob_scores = []
  for test in testing:
    img_test = io.imread(os.path.join(img_test_dir, test) , as_gray=True)
    img_test = img_test.reshape(img_test.shape[0], img_test.shape[1], 1)
    img_test_normalized = normalize_u8bits(img_test)
    x_test = window_reshape(img_test_normalized, time_steps, predictions)

    freq_test = int(test[1:test.find('_')])
    y_test = generate_sine(freq_test, x_test.shape[0]*predictions, (time_steps//2)-(predictions//2), predictions)
    
    scores = model.evaluate(x_test, y_test)
    #print(scores)
    glob_scores.append(scores)
  means = np.mean(glob_scores, axis=0)
  #print(means)
  
  return means


#-----------MAIN----------------
#dir paths
models_dir = '/content/drive/My Drive/models'
results_dir = '/content/drive/My Drive/results'
data_dir = "/content/drive/My Drive/disc_data"

model_version = 5000
model_epoch = 5

new_model_version = 5001
new_model_epoch_start = 1
new_model_epoch_quantity = 5

#network hyper-parameters, default values
time_steps = 15
predictions = 1
window_width = 320



if new_model_epoch_start == 1:
  model_name = 'model'+str(model_version)
  model_weights_name = model_name+'_epoch'+str(model_epoch)+'.h5'
else:
  model_name = 'model'+str(new_model_version)
  model_weights_name = model_name+'_epoch'+str(new_model_epoch_start-1)+'.h5'
  
model_dir = os.path.join(models_dir, model_name)
model = load_model(os.path.join(model_dir, model_weights_name))

save_model_dir = os.path.join(models_dir, 'model'+str(new_model_version))
if not os.path.isdir(save_model_dir):
  os.makedirs(save_model_dir)

  
images = os.listdir(data_dir)
for new_model_epoch in range(new_model_epoch_start, new_model_epoch_start+new_model_epoch_quantity):
  for image in images:
    if not image.endswith('.tif'):
      continue
    #read image
    groove = io.imread(os.path.join(data_dir, image), as_gray=True)
    groove = groove.reshape(groove.shape[0], groove.shape[1], 1)
    groove = normalize_u8bits(groove)

    #read track
    track_data = ET.parse(os.path.join(data_dir, image.replace('.tif', '.track')))
    root = track_data.getroot()
    track = []
    for x in root.findall('ArrayOfDouble'):
      point = []
      for d in x.findall('double'):
        point.append(int(d.text))
      track.append(point)  
    track.pop(0)

    #read sound
    samplerate, sound=read(os.path.join(data_dir, image.replace('.tif', '.wav')))
    sound = np.array(sound[0:len(track)])
    sound = normalize_16bits(sound)

    for i in range(0, len(track), 50000):
      x_data = window_reshape(groove, time_steps, predictions, track[i:min(len(track), i+50000)], window_width)
      y_data = sound[i:min(len(track), i+50000)]
      model.fit(x_data, y_data, 
                batch_size=32, 
                epochs=new_model_epoch+1, 
                initial_epoch=new_model_epoch, 
                shuffle=True,
                validation_split=0.15,
                verbose=1)
  model.save(os.path.join(save_model_dir, 'model'+str(new_model_version)+'_epoch'+str(new_model_epoch)+'.h5'))

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Train on 42500 samples, validate on 7500 samples
Epoch 2/2
42500/42500 [==============================] - 55s 1ms/step - loss: 0.0247 - mean_absolute_error: 0.1218 - mean_squared_error: 0.0247 - val_loss: 0.0142 - val_mean_absolute_error: 0.0959 - val_mean_squared_error: 0.0142
Train on 42500 samples, validate on 7500 samples
Epoch 2/2
42500/42500 [==============================] - 54s 1ms/step - loss: 0.0135 - mean_absolute_error: 0.0948 - mean_squared_error: 0.0135 - val_loss: 0.0150 - val_mean_absolute_error: 0.1023 - val_mean_squared_error: 0.0150
Train on 42500 samples, validate on 7500 samples
Epoch 2/2
42500/42500 [==============================] - 54s 1ms/step - loss: 0.0067 - mean_absolute_error: 0.0652 - mean_squared_error: 0.0067 - val_loss: 0.0029 - val_mean_absolute_error: 0.0452 - val_mean_squared_error: 0.0029
Train on 42500 samples, validate on 7500 samples
Epoch 2/2
42500/42500 [==============================] - 54s 1ms/step - loss: 0.0227 - mean_absolute_error: 0.1038

ValueError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
